In [9]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import regex as re
from pdfminer.high_level import extract_text

O objetivo é acessar o site do Congresso sobre tecnologia na Educação e baixar os pdfs dos artigos aceitos.
Para isso, a primeira etapa é se conectar com o site. Irei utilizar a biblioteca *BeautifulSoup* para isso.

In [3]:
url = 'https://sol.sbc.org.br/index.php/ctrle'

resposta = requests.get(url)

soup = BeautifulSoup(resposta.content, 'html.parser')

A próxima etapa é realizar o *scraping* (raspagem) do site. O código a seguir irá percorrer toda a tag *a* da página em html que possui no objeto *class* o nome pdf. Dessa forma é possível salvar todos os links que direcionam o pdf de cada artigo.

In [4]:
links_pdf = []
for a in soup.find_all('a', class_=re.compile('pdf'), href=True):
  if a.text:
    links_pdf.append(a['href'])

print('O site possui',len(links_pdf), 'artigos em pdf')

O site possui 75 artigos em pdf


Os links salvos são para visualização do pdf. Para realizar o download, é preciso modifica-los.

In [5]:
links_download = []
for i in links_pdf:
  links_download.append(re.sub('view', 'download', i))

print(links_download[1])

https://sol.sbc.org.br/index.php/ctrle/article/download/8871/8772


Segue função para baixar os pdf's.

In [7]:
def download(url, nome):
  resposta = requests.get(url)
  if resposta.status_code==200:
    with open(nome,"wb") as f:
      f.write(resposta.content)

Finalmente, basta chamar a função em um loop e salvar em um dataFrame as informações de cada artigo do congresso.

In [11]:
df=pd.DataFrame(columns=['text'])

for i,link in enumerate(links_download):
  pdf_name = 'pdf_{}.pdf'.format(i)
  download(link, pdf_name)
  pdf_text = extract_text(pdf_name)
  df=df.append({'text':pdf_text}, ignore_index=True)

In [17]:
df.text[0]

'IV Congresso sobre Tecnologias na Educação (Ctrl+E 2019) \n \n\nRecife, Pernambuco – Brasil \n\n28 a 30 de agosto de 2019 \n\n \n\nComo a Realidade Aumentada tem Auxiliado no Processo \nde Ensino e Aprendizagem de Ciências da Natureza? Um \nMapeamento Sistemático da Literatura \n\nEduarda Queiroz1, Rafaela Moura1, Ellen Souza1 \n\n1Universidade Federal Rural de Pernambuco – Unidade Acadêmica de Serra Talhada \n(UFRPE – UAST) - Serra Talhada, PE – Brasil \n\nmequeirozs@gmail.com, rafaela.smm98@gmail.com, ellen.ramos@ufrpe.br \n\nAbstract. Augmented Reality (AR) is defined as any system that combines the \nreal with the virtual, which is interactive in real time and is registered in three \ndimensions. In the teaching of the natural sciences it can provide students with \nthe  visualization  of  abstract  concepts.  This  work  aims  to  investigate  how  RA \nhas  aided  in  the  teaching  and  learning  process  of  the  natural  sciences.  A \nsystematic mapping was carried out, whic